# DNN

## Objetivos
- Aprender a cargar datos de un archivo CSV en un DataFrame de Pandas.
- Comprender la importancia del preprocesamiento de datos en el entrenamiento de redes neuronales.
- Crear nuestro primer modelo de una red neuronal.
- Evaluar el modelo y realizar predicicones.

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import os
import pandas as pd
import numpy as np

from IPython import display
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


## Base de datos experimental: bioreactor
El primer paso es cargar los datos desde el archivo `Experiment.csv` a un `DataFrame` utilizndo la librería **pandas**.

In [2]:
if 'google.colab' in str(get_ipython()):
  dataset_path = 'https://raw.githubusercontent.com/cursos-COnCEPT/curso-tensorflow/refs/heads/main/Experiment.csv?token=GHSAT0AAAAAAC6J2VALSXZW354GPERHUJTSZ5S7PKQ'
else:
  dataset_path = os.getcwd() + '\\Experiment.csv'

raw_dataset = pd.read_csv(dataset_path)
dataset = raw_dataset.copy()
dataset.head()

,GLC_start,feed_start,feed_end,feed_rate,VCD_start,VCD_end
0,64.552323,4,10,10.609717,0.174075,1420.0
1,25.214075,2,11,7.046333,0.869415,654.0
2,23.289962,3,8,16.556856,0.724800,1180.0
3,25.727849,3,10,19.262477,0.349741,1440.0
4,55.845139,2,10,5.105958,0.969275,842.0


En algunos casos, resulta interesante explorar los datos antendiendo a medidas estadísticas.

In [ ]:
dataset.describe()

Así como representar los datos de forma gráfica.

In [ ]:
for input_feature in dataset.columns:
    if input_feature != 'VCD_end':
        dataset.plot.scatter(x=input_feature, y='VCD_end')
        plt.xlabel(input_feature)
        plt.ylabel('VCD_end')
        plt.show()

## Preprocesamiento de datos
### 1 - División o *train-test-validation split*

Nuestro objetivo es crear un modelo que generalice bien a los datos no vistos y no simplemente que se ajuste perfectamente a algunos datos vistos. Por este motivo debemos asegurar que al dividir los datos, los conjunto de `test`sea lo más parecido a los datos futuros en los que se quiere aplicar el modelo. 

En este caso, basta con dividir nuestro `dataset` de forma aleatoria. Sin embargo, hay situaciones más complejas en las que hay que tener cuidado con cómo se hace el reparto. Puedes aprender más sobre el *stratified splitting* [aquí](https://scikit-learn.org/stable/modules/cross_validation.html#stratification).

In [ ]:
train_ratio = .7
test_ratio = .15
val_ratio = .15

X_train = dataset.sample(frac=train_ratio+val_ratio, random_state=0)
X_test = dataset.drop(X_train.index)
X_val = X_train.sample(frac=val_ratio/(val_ratio+train_ratio), random_state=0)

### 2- Separación inputs y outputs

In [ ]:
y_train = X_train.pop('VCD_end')
y_test = X_test.pop('VCD_end')
y_val = X_val.pop('VCD_end')

### 3 - Escalado 
El análisis estadístico ha sido útil para identificar cómo las variables de entrada tienen diferentes escalas, lo cual puede afectar al proceso de entrenamiento, ya que las variables que se mueven en un rango más amplio tienen mayor impacto sobre el modelo.

Para evitar que esto ocurra, es recomendable escalar los datos. Para este ejemplo, vamos a utilizar la función `MinMaxScaler` del paquete **sklearn**. Internamente, esta función realiza la siguiente operación:
```python
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
```

In [ ]:
from sklearn.preprocessing import MinMaxScaler

Primero inicializamos nuestra función de escalado y después llamamos al comando `fit` para obtener los parámetros de la función, esto es, los valores mínimo y máximo de cada variable de entrada en el training set.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

**Atención:** el escalado no se aplica a los outputs. ([*Should you scale target values in a regression problem?*](https://www.kaggle.com/discussions/questions-and-answers/181908)).

A continuación, escalamos los datos.

In [ ]:
X_train_nondim = scaler.transform(X_train)
X_test_nondim = scaler.transform(X_test)
X_val_nondim = scaler.transform(X_val)

In [ ]:
X_train.describe()

#### Ejercicio - leer la documentación
Para aprender sobre *machine learning* o programación en general, es fundamental saber manejarse con la documentación de las diferentes herramientas y extraer la información que nos interesa.

Visita la [web de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) y contesta la siguiente pregunta:
- ¿Cuáles son los valores de `X.min` y `X.max` que se utilizan ara escalar los datos?

In [ ]:
# Completa con tu código aquí
# min_values = 
# max_values = 

# print('Los valores mínimos y máximos para cada variable son:')
for i, col in enumerate(X_train.columns):
    print(f'{col}: {min_values[i]} - {max_values[i]}')

Es importante conocer qué valores se han utilizado para hacer el escalado de los datos de entrenamiento porque luego deberán aplicarse los mismos a cualquier otro dato que se alimente al modelo.

## Modelo

### 1 - Definición del modelo
Aquí vamos a utilizar un modelo secuencial con 3 capas internas con 128, 64 y 32 neuronas, respectivamente, seguidas de una capa de salida que devuelve un único valor, el de la concentración de VCD al final del experimento. Para las capas internas, aplica funciones de activación tipo ReLU. En problemas de regresión, no suele aplicarse ninguna función de activación en la *output layer*.

Aunque no es estrictamente necesario, vamos a agrupar todos los pasos de la construcción del modelo como una función (se llamará `build_model`).

In [ ]:
num_inputs = X_train.shape[1]
num_hidden_neurons = [128, 64, 32]
num_outputs = 1

EPOCHS = 200
BATCH_SIZE = 10
N_TRAIN = X_train.shape[0]
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

initial_learning_rate = 0.01

La mayoría de modelos convergen mejor cuando se disminuye el *learning rate* de forma gradual durante el entrenamiento. 

Para modular cómo varía el *learning rate* se utilizan funciones llamadas *learning rate schedules*. Existen diversos *schedules* ya implementados en TensorFloW, entre los que vamos a utilizar [InverseTimeDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/InverseTimeDecay).


In [ ]:
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate,
    decay_steps=STEPS_PER_EPOCH*50,
    decay_rate=1,
    staircase=True)

El código de arriba crea un `tf.keras.optimizers.schedules.InverseTimeDecay` para disminuir el *learning rate* a la mitad pasadas las primeras 50 épocas, 1/3 a las 100, etc.

In [ ]:
step = np.linspace(0,EPOCHS*STEPS_PER_EPOCH)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

In [ ]:
def build_model(num_hidden_neurons, lr):
    layers_list = []
    layers_list.append(layers.InputLayer(shape=[len(X_train.columns)],
                                         name = 'INPUT'))
    for i in range(len(num_hidden_neurons)):
        layers_list.append(layers.Dense(num_hidden_neurons[i], 
                                        activation='relu', 
                                        kernel_initializer='he_normal', 
                                        name = f'HIDDEN_{i}'))
    layers_list.append(layers.Dense(num_outputs, 
                        activation = 'linear', 
                        name = 'OUTPUT'))

    model = keras.Sequential(layers_list)
    
    optimizer = tf.keras.optimizers.Adam(lr)

    model.compile(  loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
    return model

Una vez construido, el método `summary` permite inspeccionar el modelo.

In [ ]:
model = build_model(num_hidden_neurons, lr_schedule)
model.summary()

Teóricamente, ya podríamos utilizar el modelo para hacer predicciones.

In [ ]:
example_batch = X_train_nondim[:10]
example_result = model.predict(example_batch)

results_df = pd.DataFrame({
    'Real Values': y_train[:10].values,
    'Predicted Values': example_result.flatten()
})
print(results_df)

### 2 - Entrenamiento
Aunque el modelo parece funcionar y da un resultado de la forma y tipo esperados, las predicciones no se ajustan a los valores esperados. Esto se debe a que todavía no está entrenado, es decir, hay que optimizar los valors de los pesos o `weights` de la red neuronal.

Entrena el modelo durante 200 épocas y representa la curva de entrenamiento.

In [ ]:
history = model.fit(X_train_nondim, y_train,
        epochs=EPOCHS, 
        batch_size = BATCH_SIZE,
        validation_data = (X_val_nondim, y_val),
        verbose=1,
        callbacks=[])

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [UNITS]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$UNITS^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Validation Error')
  plt.legend()
  plt.show()


plot_history(history)

### 3 - Evaluación

In [ ]:
loss, mae, mse = model.evaluate(X_test_nondim, y_test, verbose=2)

print("MAE en test: {:5.2f} UNITS".format(mae))

### 4 - Realizar predicciones

In [ ]:
test_predictions = model.predict(X_test_nondim).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('Real Value')
plt.ylabel('Predicted Value')
p1 = max(max(test_predictions), max(y_test))
p2 = min(min(test_predictions), min(y_test))
plt.plot([p1, p2], [p1, p2], '--k')
plt.xlim([p1,p2])
plt.ylim([p1,p2])
plt.axis('square')

También podemos visualizar la distribución de errores.

In [ ]:
error = test_predictions - y_test
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [UNITS]")
_ = plt.ylabel("Count")

## Conclusiones

Con este ejemplo hemos aprendido:

* El error cuadrático medio (MSE) es una *loss function* muy utilizada para problemas de regresión (se utilizan diferentes funciones de pérdida para problemas de clasificación).
* Otra métrica muy común para evaluar el modelo es el error absoluto medio (MAE).
* Cuando las *inputs* tienen valores con diferentes rangos, deben aplicarse técnicas de escalado.

Con este modelo se va a aprender a crear una red neuronal
con otro ejemplo se va a ver el overfitting + early stopping
con otro diferente el hyperparam tunning
se vuelve a este si da tiempo para el concurso